In [1]:
# Imports
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import trackml.dataset
from trackml.utils import add_momentum_quantities
import matplotlib.pyplot as plt
import numpy as np


/local/anaconda3/envs/gnn4itk/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')
print(device)

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

cuda:7


In [ ]:
# Data Visualization
event_path = "/home/atlas/dittmeier/public_html/pytorch/trackml_data/trainset/event000021000"

particles, hits, cells, truth = trackml.dataset.load_event(
    event_path, parts=["particles", "hits", "cells", "truth"]
)

print(truth.head())
print(truth["particle_id"].value_counts())

#print(count.item())


# Create histograms for particles
plt.figure(figsize=(12, 8))
for i, column in enumerate(particles.columns):
    plt.subplot(2, 5, i+1)
    plt.hist(particles[column], bins=50)
    plt.title(column)
plt.tight_layout()
plt.show()

# Create histograms for hits
plt.figure(figsize=(12, 8))
for i, column in enumerate(hits.columns):
    plt.subplot(2, 4, i+1)
    plt.hist(hits[column], bins=50)
    plt.title(column)
plt.tight_layout()
plt.show()

# Create histograms for cells
plt.figure(figsize=(12, 8))
for i, column in enumerate(cells.columns):
    plt.subplot(2, 3, i+1)
    plt.hist(cells[column], bins=50)
    plt.title(column)
plt.tight_layout()
plt.show()

# Create histograms for truth
plt.figure(figsize=(12, 8))
for i, column in enumerate(truth.columns):
    plt.subplot(2, 5, i+1)
    plt.hist(truth[column], bins=50)
    plt.title(column)
plt.tight_layout()
plt.show()


In [3]:
# Create a dataset, with a label for each hit, if hits have same particle_id set to 1, else -1
# and an option to apply a cut on pT on the dataset

class TrackMLDataset(Dataset):
    def __init__(self, event_path, nevents=10, cuts=None):
        self.event_path = event_path
        self.nevents = nevents
        self.event_ids = []
        self.particles = []
        self.hits = []
        self.cells = []
        self.truth = []
        self.labels = []

        for event_id, hits, cells, particles, truth in trackml.dataset.load_dataset(event_path, nevents=nevents):
            particles = add_momentum_quantities(particles)
            print(event_id)
            if cuts is not None:
                print(f"Applying cuts: {cuts}")
                particles = particles[particles.pt >= cuts["pt"]].reset_index(drop=True)
                truth = truth[truth.particle_id.isin(particles.particle_id)].reset_index(drop=True)
                hits  = hits[hits.hit_id.isin(truth.hit_id)].reset_index(drop=True)
                cells = cells[cells.hit_id.isin(hits.hit_id)].reset_index(drop=True)
                
            self.event_ids.append(event_id)
            self.particles.append(particles)
            self.hits.append(hits)
            self.cells.append(cells)
            self.truth.append(truth)
            # create a label for a matrix of shape (n_hits, n_hits), if hits have same particle_id set to 1, else -1
            # we can be more sophisticated here, but for now this is enough
            labels = np.full((len(hits), len(hits)), -1, dtype=np.int8)    # this is not memory friendly!!!
            for particle_id, nhits in zip(particles.particle_id, particles.nhits):
                particle_hits = truth[truth.particle_id == particle_id].index
                #meshgrid = np.meshgrid(particle_hits, particle_hits)
                #print(f"particle_id: {particle_id}, nhits = {nhits}, particle_hits = {particle_hits}, meshgrid = {meshgrid}")
                #labels[meshgrid] = 1
                for match in particle_hits:
                    for match2 in particle_hits:
                        labels[match, match2] = 1
#            print(labels.size)
            #if np.array_equal(labels, labels2):
            #    print("Labels are equal")
            #count_1 = np.count_nonzero(labels == 1)
            #print(f"Count of 1 in labels: {count_1}")
            #count_2= np.count_nonzero(labels2 == 1)
            #print(f"Count of 1 in labels2: {count_2}")
            #labels = np.full((1,1),0)
            self.labels.append(labels)

    def __len__(self):
        return len(self.event_ids)

    def __getitem__(self, idx):
        event_id = self.event_ids[idx]
        particles = self.particles[idx]
        hits = self.hits[idx]
        cells = self.cells[idx]
        truth = self.truth[idx]
        labels = self.labels[idx]
        return event_id, torch.tensor(particles.values, dtype=torch.float32), torch.tensor(hits.values, dtype=torch.float32), torch.tensor(cells.values, dtype=torch.float32), torch.tensor(truth.values, dtype=torch.float32), torch.from_numpy(labels).type(torch.int8)


In [4]:
print("Loading Trainset")
event_path = "/home/atlas/dittmeier/public_html/pytorch/trackml_data/trainset"
training_data = TrackMLDataset(event_path, nevents=40, cuts={"pt": 1})
print("Loading Valset")
event_path = "/home/atlas/dittmeier/public_html/pytorch/trackml_data/valset"
val_data = TrackMLDataset(event_path, cuts={"pt": 1})
print("Loading Testset")
event_path = "/home/atlas/dittmeier/public_html/pytorch/trackml_data/testset"
test_data = TrackMLDataset(event_path, cuts={"pt": 1})

Loading Trainset
21000
Applying cuts: {'pt': 1}
21001
Applying cuts: {'pt': 1}
21002
Applying cuts: {'pt': 1}
21003
Applying cuts: {'pt': 1}
21004
Applying cuts: {'pt': 1}
21005
Applying cuts: {'pt': 1}
21006
Applying cuts: {'pt': 1}
21007
Applying cuts: {'pt': 1}
21008
Applying cuts: {'pt': 1}
21009
Applying cuts: {'pt': 1}
21010
Applying cuts: {'pt': 1}
21011
Applying cuts: {'pt': 1}
21012
Applying cuts: {'pt': 1}
21013
Applying cuts: {'pt': 1}
21014
Applying cuts: {'pt': 1}
21015
Applying cuts: {'pt': 1}
21016
Applying cuts: {'pt': 1}
21017
Applying cuts: {'pt': 1}
21018
Applying cuts: {'pt': 1}
21019
Applying cuts: {'pt': 1}
21020
Applying cuts: {'pt': 1}
21021
Applying cuts: {'pt': 1}
21022
Applying cuts: {'pt': 1}
21023
Applying cuts: {'pt': 1}
21024
Applying cuts: {'pt': 1}
21025
Applying cuts: {'pt': 1}
21026
Applying cuts: {'pt': 1}
21027
Applying cuts: {'pt': 1}
21028
Applying cuts: {'pt': 1}
21029
Applying cuts: {'pt': 1}
21030
Applying cuts: {'pt': 1}
21031
Applying cuts: {

In [5]:
batch_size = 1
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

event_id, particles, hits, cells, truth, labels = next(iter(train_dataloader))
#event_id, particles, hits, cells, truth = next(iter(train_dataloader))
print(f"event_id shape: {event_id.size()}; event_id data type: {event_id.dtype}")
print(f"particles shape: {particles.size()}; particles data type: {particles.dtype}")
print(f"hits shape: {hits.size()}; hits data type: {hits.dtype}")
print(f"cells shape: {cells.size()}; cells data type: {cells.dtype}")
print(f"truth shape: {truth.size()}; truth data type: {truth.dtype}")
#print(f"labels shape: {labels.size()}; labels data type: {labels.dtype}")

print(f"event_id: {event_id.squeeze()}")
print(f"hit coordinates: {hits.squeeze()[:,1:4]}")    # x, y, z
#print(f"labels: {labels.squeeze()}")



event_id shape: torch.Size([1]); event_id data type: torch.int64
particles shape: torch.Size([1, 905, 14]); particles data type: torch.float32
hits shape: torch.Size([1, 9685, 7]); hits data type: torch.float32
cells shape: torch.Size([1, 29479, 4]); cells data type: torch.float32
truth shape: torch.Size([1, 9685, 9]); truth data type: torch.float32
event_id: 21041
hit coordinates: tensor([[  -74.6183,    -8.1546, -1502.5000],
        [  -54.1529,   -27.5452, -1502.0000],
        [ -111.3620,   -51.9861, -1502.0000],
        ...,
        [ -867.1360,   488.6900,  2952.5000],
        [ -904.3210,   298.8160,  2952.5000],
        [ -754.5060,   131.1770,  2947.5000]])


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()       
        self.linear_relu_stack = nn.Sequential( # Sequential container
            nn.Linear(3, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 8),
        )
        
    def forward(self, x):
        y = self.linear_relu_stack(x)
        return y

# sends the model to GPU
model = NeuralNetwork().to(device)

In [7]:
learning_rate = 1e-3
batch_size = batch_size
epochs = 1
margin = 1.0

loss_fn = nn.HingeEmbeddingLoss(margin=margin)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
#test run
model.eval()
result = model(hits.squeeze()[:,1:4].to(device))
print(result.size())
# calculate distances between all hits
distances = torch.cdist(result, result)
print(distances.size())
print(distances)


In [ ]:
# we want to do a knn from scikit
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=20)
neigh.fit(result.detach().numpy())
distances, indices = neigh.kneighbors(result.detach().numpy())
#print(indices)
positives = 0
negatives = 0

count_1 = torch.sum(labels == 1).item()
print(f"Count of 1 in labels: {count_1}")

for ind in indices:
    # we can just check with labels, if knn and label = 1 --> match, -1 --> bad
    #print(ind)
    positives += np.sum((labels.squeeze()[ind[0],ind[1:]] == 1).numpy())
    negatives += np.sum((labels.squeeze()[ind[0],ind[1:]] == -1).numpy())
    #print(f"positives: {positives}; negatives: {negatives}")
    
print(f"positives: {positives}; negatives: {negatives}")

In [8]:
# loops over our optimization code
def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    #print(f"size = {size}")
    for batch, [event_id, particles, hits, cells, truth, labels] in enumerate(dataloader):
        # Send data to GPU
        particles, hits, cells, truth, labels = to_device([particles, hits, cells, truth, labels], device)
        
        embedding = model(hits.squeeze()[:,1:4])
        distances = torch.cdist(embedding, embedding)        

        # creating labels here is time consuming.
        
        #labels = torch.ones((hits.squeeze().size(dim=0), hits.squeeze().size(dim=0)), dtype=torch.int8)*(-1)    # this is not memory friendly!!!
        #labels = labels.to(device)

        #for particle_id, nhits in zip (particles.squeeze()[:,0], particles.squeeze()[:,9]):
        #    truth = truth.squeeze()
        #    mask = truth[:,1] == particle_id
            #print(mask)
            #count_trues = torch.sum(mask).item()
            #print(f"Count of True values in mask: {count_trues}, compare to nhits :{nhits}")
        #    true_positions = mask.nonzero().flatten()
            #print(true_positions)
        #    for match in true_positions:
        #       for match2 in true_positions:
        #           labels[match, match2] = 1
        #count_ones = torch.sum(labels == 1).item()
        #print(f"Count of ones in labels: {count_ones}")        
        loss = loss_fn(distances, labels)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 1 == 0:
            loss, current = loss.item(), batch * batch_size + len(event_id)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

        # Clear GPU memory
        del event_id, particles, hits, cells, truth, embedding, distances, labels, loss
        torch.cuda.empty_cache()



In [9]:
# evaluate the model's performance against the test dataset
def test_loop(dataloader, model, loss_fn, device):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, sum_of_residuals = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for event_id, particles, hits, cells, truth, labels  in dataloader:
            # Send data to GPU
            particles, hits, cells, truth, labels = to_device([particles, hits, cells, truth, labels], device)
            
            embedding = model(hits.squeeze()[:,1:4])
            distances = torch.cdist(embedding, embedding)

            #labels = torch.ones((hits.squeeze().size(dim=0), hits.squeeze().size(dim=0)), dtype=torch.int8)*(-1)    # this is not memory friendly!!!
            #labels = labels.to(device)

            #for particle_id, nhits in zip (particles.squeeze()[:,0], particles.squeeze()[:,9]):
            #    truth = truth.squeeze()
            #    mask = truth[:,1] == particle_id
            #    #print(mask)
            #    #count_trues = torch.sum(mask).item()
            #    #print(f"Count of True values in mask: {count_trues}, compare to nhits :{nhits}")
            #    true_positions = mask.nonzero().flatten()
            #    #print(true_positions)
            #    for match in true_positions:
            #        for match2 in true_positions:
            #            labels[match, match2] = 1
            ##count_ones = torch.sum(labels == 1).item()
            ##print(f"Count of ones in labels: {count_ones}")


            test_loss += loss_fn(distances, labels).item()
            del event_id, particles, hits, cells, truth, embedding, distances, labels

    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")
    return test_loss

In [10]:
val_loss = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, device)
    val_loss.append(test_loop(val_dataloader, model, loss_fn, device))
print("Done!")


Epoch 1
-------------------------------
loss: 0.114261  [    1/   50]
loss: 0.132981  [    2/   50]
loss: 0.125559  [    3/   50]
loss: 0.099772  [    4/   50]
loss: 0.132228  [    5/   50]
loss: 0.092991  [    6/   50]
loss: 0.113123  [    7/   50]
loss: 0.077410  [    8/   50]
loss: 0.089579  [    9/   50]
loss: 0.076422  [   10/   50]
loss: 0.066747  [   11/   50]
loss: 0.074070  [   12/   50]
loss: 0.057488  [   13/   50]
loss: 0.069394  [   14/   50]
loss: 0.066744  [   15/   50]
loss: 0.062766  [   16/   50]
loss: 0.046690  [   17/   50]
loss: 0.044129  [   18/   50]
loss: 0.038634  [   19/   50]
loss: 0.035923  [   20/   50]
loss: 0.036586  [   21/   50]
loss: 0.030957  [   22/   50]
loss: 0.026448  [   23/   50]
loss: 0.051384  [   24/   50]
loss: 0.026868  [   25/   50]
loss: 0.034084  [   26/   50]
loss: 0.024167  [   27/   50]
loss: 0.025820  [   28/   50]
loss: 0.024173  [   29/   50]
loss: 0.024144  [   30/   50]
loss: 0.026942  [   31/   50]
loss: 0.019273  [   32/   50]


In [ ]:
#for name, param in model.named_parameters():
#    print(f"Parameter name: {name}, Size: {param.size()}, Values: {param}")

import matplotlib.pyplot as plt
# Assuming you have the `test_loss` variable containing the loss values for each epoch
epochs = range(1, len(test_loss) + 1)
plt.plot(epochs, test_loss, 'b', label='Test Loss')
plt.title('Test Loss vs Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
array = np.random.uniform(-1,1,size=(50, Ndims))
model.eval()
output = model(torch.tensor(array).float())
function_without_noise = np.zeros(50)
for i, x in enumerate(array):
    function_without_noise[i] = np.sum(coefficients*x)
#print(function_without_noise)
for i in range(5):
    print(array[i], output[i])
plt.plot(array[:,0], output.detach().numpy(), 'ro', label='Model Output')
plt.plot(array[:,0], function_without_noise, 'b*', label='Function without Noise')
plt.xlabel('Feature0')
plt.ylabel('Output')
plt.title('Model Output vs Function without Noise')
plt.legend()
plt.show()